In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q torch datasets
!pip install -q accelerate==0.21.0 \
                peft==0.4.0  \
                bitsandbytes==0.40.2  \
                transformers==4.31.0  \
                trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 48.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0


In [ ]:
!nvidia-smi

Mon Mar 25 16:12:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)

In [ ]:
import pandas as pd

dataset = pd.read_csv('/content/drive/MyDrive/llm-code-review-clj/Dataset/com-filtro/filtered_by_number_of_lines.csv')
dataset.head()

,pull_request_url,comment_id,diff_hunk,content,cleaned_diff_hunk,diff_lines
0,https://api.github.com/repos/adamtornhill/code...,33513420,"@@ -1,3 +1,5 @@+[![Build Status](https://travi...",Ah sorry there is clearly an issue here @adamt...,[![Build Status](https:,1
1,https://api.github.com/repos/adamtornhill/code...,95127516,"@@ -10,52 +10,106 @@ ""/some/path => G1 /anothe...",The test cases are a bit too noisy. We need to...,"(def ^:const multi-regexp-group-spec\n""^/some/...",9
2,https://api.github.com/repos/anmonteiro/lumo/p...,87304994,"@@ -35,7 +35,7 @@ Check out `lumo -h` for usag...",:+1:,"2. At the root of the repository, run: `boot r...",1
3,https://api.github.com/repos/anmonteiro/lumo/p...,88037369,"@@ -0,0 +1,35 @@+import net from 'net';",this needs a `/* @flow */` header,import net from 'net';,1
4,https://api.github.com/repos/anmonteiro/lumo/p...,88787684,"@@ -75,30 +76,35 @@ function consumeBuffer(buf...",can we keep the readline interface as the firs...,"export function processLine(sessionId: number,...",1


In [ ]:
_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/llm-code-review-clj/Modelos/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/llm-code-review-clj/Modelos/Mistral-7B-v0.1",
    trust_remote_code=True,
    device_map="auto",
    load_in_8bit=True,
    max_memory={0:"15GB"}
)

model.eval()
model = torch.compile(model, mode = "max-autotune", backend = "inductor")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
##CUIDADO PARA NÃO RODAR DE NOVO

headers = ['pull_request_url', 'diff_hunk', 'model_output']
empty_df = pd.DataFrame(columns=headers)
empty_df.to_csv('/content/drive/MyDrive/llm-code-review-clj/Dataset/output/default_mistral_output(2).csv', index=False)

In [ ]:
existing_data = pd.read_csv('/content/drive/MyDrive/llm-code-review-clj/Dataset/output/default_mistral_output(2).csv')

i = 0
for index, row in dataset.iterrows():
   if i == 50: break
   if row['diff_lines'] >= 5:
    code_snippet = row['diff_hunk']
    prompt = f"""
              <s>[INST] <<SYS>>
              If a piece of code doesn't make sense or isn't factually coherent, explain why rather than answering something incorrect.
              If you don't know the answer to a question, don't share false information.
              <</SYS>>
              Given that you are an experienced programmer, review the code snippet below:

              {code_snippet}

              Format your review with a list of concrete instructions to the author of the code.
              [/INST]
              """
    device = 'cuda'
    model_inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    generated_ids = model.generate(**model_inputs, temperature=0.1, top_k=1, top_p=1.0, repetition_penalty=1.4, min_new_tokens=16, max_new_tokens=128, do_sample=True)
    decoded = tokenizer.decode(generated_ids[0])

    answer = decoded.split(prompt)[-1]
    answer = answer.replace("<s>", "").replace("</s>", "").replace("<pad>", "")

    data_list = []
    new_data = {
          'pull_request_url': row['pull_request_url'],
          'diff_hunk': row['diff_hunk'],
          'model_output': answer
      }
    data_list.append(new_data)

    columns = ['pull_request_url', 'diff_hunk', 'model_output']
    new_data_df = pd.DataFrame(data_list, index=range(1, len(data_list) + 1), columns=columns)

    existing_data = existing_data.append(new_data, ignore_index=True)
    existing_data.to_csv('/content/drive/MyDrive/llm-code-review-clj/Dataset/output/default_mistral_output.csv', index=False)

    print(f"{i+1}/100 trecho de código analisado")
    i+=1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
<ipython-input-16-8d3839ab8437>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(new_data, ignore_index=True)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1/100 trecho de código analisado


<ipython-input-16-8d3839ab8437>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(new_data, ignore_index=True)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2/100 trecho de código analisado


<ipython-input-16-8d3839ab8437>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(new_data, ignore_index=True)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

<ipython-input-16-8d3839ab8437>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(new_data, ignore_index=True)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


48/100 trecho de código analisado


<ipython-input-16-8d3839ab8437>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(new_data, ignore_index=True)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


49/100 trecho de código analisado
50/100 trecho de código analisado


<ipython-input-16-8d3839ab8437>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(new_data, ignore_index=True)
